In [1]:
import tensorflow as tf
from keras import layers
from keras import preprocessing
import word2vec as w2v
import dataProsessing as dp
import numpy as np
from tensorboard.plugins import projector
import os
import io
import json

In [4]:

#run this once and only once
#os.chdir("../")
print(os.getcwd())

C:\Users\Starapter\GitBash\MAT_180_ML_Projects\corpus-of-chords


In [15]:

%tensorboard --logdir logs\embeddings


UsageError: Line magic function `%tensorboard` not found.


In [12]:
datasetName = "Jsb16thSeparated(t_60_rr_re)"

corpus = dp.loadDataset("Data\\Corpi\\" + datasetName + "\\" + datasetName +".json")
vocab = dataset = json.load(open("Data\\Corpi\\" + datasetName + "\\" + datasetName +"_vocab.json"))
inv_vocab = vocab["inv_vocab"]
vocab = vocab["vocab"]

In [14]:
BATCH_SIZE = 1024
BUFFER_SIZE = 10000
embedding_dim = 128
negative_samples = 4
log_dir = "logs\embeddings"

vectorizedSequences = dp.vectorizeDataset(corpus,vocab)
targets,contexts,labels = [],[],[]
for s in vectorizedSequences:
    tmp = w2v.generateTrainingData(s,vocab_size=len(vocab),window_size=1,negative_samples=negative_samples)
    targets += tmp[0]
    contexts += tmp[1]
    labels += tmp[2]
    
targets = np.array(targets)
contexts = np.array(contexts)
labels = np.array(labels)

dataset = tf.data.Dataset.from_tensor_slices(((targets,contexts),labels))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)


word2vec = w2v.Word2Vec(len(vocab), embedding_dim,4)
word2vec.compile(optimizer='adam', loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")
word2vec.fit(dataset, epochs=20, callbacks=[tensorboard_callback])

weights = tf.Variable(word2vec.get_layer('w2v_embedding').get_weights()[0])

checkpoint = tf.train.Checkpoint(embedding=weights)
checkpoint.save(os.path.join(log_dir, "embedding.ckpt"))

dp.generateMetadata(dir = log_dir, inv_vocab = inv_vocab)

out_m = io.open(os.path.join(log_dir, 'metadata.tsv'), 'w', encoding='utf-8')

for i in range(len(inv_vocab.items())):
    out_m.write(str(inv_vocab[i]) + "\n")

out_m.close()


config = projector.ProjectorConfig()
embedding = config.embeddings.add()
# The name of the tensor will be suffixed by `/.ATTRIBUTES/VARIABLE_VALUE`.
embedding.tensor_name = "embedding/.ATTRIBUTES/VARIABLE_VALUE"
embedding.metadata_path = 'metadata.tsv'
projector.visualize_embeddings(log_dir, config)




KeyError: (60, 55, 52, 48)